# MTM4511 - 19052013
# BÖLÜM 4.2 KODLARI

## New York Blizzard Videosu

Bu kısımda video'nun path'i doğru verilmezse kod hata verebilir. Kod farklı bir bilgisayarda çalıştırıldığında path değişebilir! 

**Klavyede q tuşuna basılması dahilinde video kapanacaktır!**

In [2]:
video_path = r'C:\CODING\19052013_MMTU_CD\ObjectDetection-YOLOv8-SSDMobileNet\data\original-video-newyork-blizzard.mp4'

In [16]:
import cv2

def play_video(video_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    total_seconds = total_frames / fps

    cv2.namedWindow('Video Player', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Video Player', width, height)

    start_time = cv2.getTickCount()

    while True:
        ret, frame = cap.read()

        if not ret:
            print("End of video")
            break

        elapsed_time = (cv2.getTickCount() - start_time) / cv2.getTickFrequency()
        current_seconds = int(elapsed_time)
        remaining_seconds = int(total_seconds - current_seconds)
        display_text = f"{current_seconds:.1f}/{total_seconds:.1f} - FPS: {fps:.1f}"

        cv2.putText(frame, display_text, (width - 320, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('Video Player', frame)

        if cv2.waitKey(int(1000 / fps)) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Videoyu oynat
play_video(video_path)


Sağ üst köşede gözlemlenebileceği üzere video sabit 30 FPS'ye (frame per second) sahiptir.

### **YOLOv8** Object Detection Performansı - New York Blizzard Video**

Performans sırasında görsel karşılaştırmaları yapabilmek adına 5 saniyede bir ekran görüntüsü alan kod satırları eklenmiştir.

In [4]:
from ultralytics import YOLO
import cv2
import math
import torch
import time
import os
from datetime import datetime



screen_shot_path = r'C:\CODING\Python.PROJECTS\19052013_MMTU_CD\ObjectDetection-YOLOv8-SSDMobileNet\ScreenShots\NewYork-YOLOv8'



cap = cv2.VideoCapture(video_path)
cap.set(3, 2000)
cap.set(4, 1000)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model
model = YOLO("yolo-Weights/yolov8n.pt")
model.to(device)

# object classes
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"]

# Initialize variables for FPS calculation and screenshot timing
start_time = time.time()
frames_processed = 0
screenshot_interval = 5  # İstenilen her kaç saniyede bir ekran görüntüsü alınacağını belirtir
screenshot_count = 0
screenshot_time = 0


while True:
    success, img = cap.read()

    if not success:
        print("Error reading video stream.")
        break

    # Perform YOLO object detection
    results = model(img, stream=True)

    # Calculate FPS
    frames_processed += 1
    elapsed_time = time.time() - start_time
    fps = frames_processed / elapsed_time

    # Display FPS on the frame
    cv2.putText(img, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Draw bounding boxes and class names
    for r in results:
        boxes = r.boxes

        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

            confidence = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])

            org = [x1, y1]
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = (255, 0, 0)
            thickness = 2

            cv2.putText(img, classNames[cls], org, font, fontScale, color, thickness)

    # Display the frame
    cv2.imshow("YOLO Object Detection", img)

    # Check if it's time to take a screenshot
    if time.time() - screenshot_time >= screenshot_interval:
        screenshot_time = time.time()

        # Create the screenshot path if it doesn't exist
        if not os.path.exists(screen_shot_path):
            os.makedirs(screen_shot_path)

        # Generate a filename for the screenshot using the current timestamp and count
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        screenshot_filename = f"screenshot_{timestamp}_{screenshot_count}.png"
        screenshot_full_path = os.path.join(screen_shot_path, screenshot_filename)

        # Save the screenshot
        cv2.imwrite(screenshot_full_path, img)
        print(f"Screenshot saved: {screenshot_full_path}")

        # Increment the screenshot count
        screenshot_count += 1

    if cv2.waitKey(1) == ord('q'):
        break


# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()



0: 384x640 9 persons, 2 cars, 2 traffic lights, 27.7ms
Speed: 3.2ms preprocess, 27.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Screenshot saved: C:\CODING\19052013_MMTU_CD\ObjectDetection-YOLOv8-SSDMobileNet\ScreenShots-YOLOv8\screenshot_20240103144937_0.png

0: 384x640 10 persons, 2 cars, 1 traffic light, 23.3ms
Speed: 6.3ms preprocess, 23.3ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 2 cars, 1 bus, 1 traffic light, 25.3ms
Speed: 4.0ms preprocess, 25.3ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 2 cars, 2 traffic lights, 23.7ms
Speed: 10.6ms preprocess, 23.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 2 cars, 23.2ms
Speed: 3.4ms preprocess, 23.2ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 2 cars, 3 traffic lights, 23.7ms
Speed: 3.4ms preprocess, 23.7ms inference, 1.0ms postproce

### **SSD Mobile Net** Object Detection Performansı - New York Blizzard Video

Performans sırasında görsel karşılaştırmaları yapabilmek adına 5 saniyede bir ekran görüntüsü alan kod satırları eklenmiştir.

In [7]:
import cv2
import time
import numpy as np

cap = cv2.VideoCapture(video_path)
cap.set(3, 1280) # 3 represents width 
cap.set(4, 720) # 4 represents hight
cap.set(10, 70) # 10 represents brightness

threshold = 0.5
nms_threshold = 0.1

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
total_seconds = total_frames / fps

start_time = cv2.getTickCount()  

classNames = []
classFile = r'ssd-nesne-tanima-main\ssd-nesne-tanima-main\coco.names'
with open(classFile, 'rt') as f:
    classNames = f.read().rsplit('\n')

# print(classNames) # classNames can be observed
    

configPath = r'ssd-nesne-tanima-main\ssd-nesne-tanima-main\ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
weightsPath = r'ssd-nesne-tanima-main\ssd-nesne-tanima-main\frozen_inference_graph.pb'

SSDMobileNet = cv2.dnn_DetectionModel(weightsPath, configPath)
SSDMobileNet.setInputSize(320, 320)
SSDMobileNet.setInputScale(1.0/127.5)
SSDMobileNet.setInputMean((127.5, 127.5, 127.5))
SSDMobileNet.setInputSwapRB(True)


# Initialize variables for FPS calculation and screenshot timing
start_time = time.time()
frames_processed = 0
screenshot_interval = 5  # İstenilen her kaç saniyede bir ekran görüntüsü alınacağını belirtir
screenshot_count = 0
screenshot_time = 0

while True:
    success, frame = cap.read()

    if not success:
        print("Error reading video stream.")
        break

    current_time = time.time()
    elapsed_time = current_time - start_time
    current_seconds = int(elapsed_time)

    fps_and_videoLen_text = f"FPS: {frames_processed / elapsed_time:.2f} | Time: {current_seconds:.1f}/{total_seconds:.1f}"

    cv2.putText(frame, fps_and_videoLen_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    classIds, confs, bbox = SSDMobileNet.detect(frame, confThreshold=threshold)
    bbox = list(bbox)
    confs = list(np.array(confs).reshape(1, -1)[0])
    confs = list(map(float, confs))

    indices = cv2.dnn.NMSBoxes(bbox, confs, threshold, nms_threshold)

    for i in indices:
        box = bbox[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        cv2.rectangle(frame, (x, y), (x + w, h + y), color=(0, 255, 0), thickness=2)
        cv2.putText(frame, classNames[classIds[i] - 1].upper(), (box[0] + 10, box[1] + 30),
                    cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("SSD Mobile Net Object Detection", frame)

    # Check if it's time to take a screenshot
    if current_time - screenshot_time >= screenshot_interval:
        screenshot_time = current_time

        # Create the screenshot path if it doesn't exist
        screen_shot_path = r'C:\CODING\Python.PROJECTS\19052013_MMTU_CD\ObjectDetection-YOLOv8-SSDMobileNet\ScreenShots\NewYork-SSDMobileNet'
        if not os.path.exists(screen_shot_path):
            os.makedirs(screen_shot_path)

        # Generate a filename for the screenshot using the current timestamp and count
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        screenshot_filename = f"screenshot_{timestamp}_{screenshot_count}.png"
        screenshot_full_path = os.path.join(screen_shot_path, screenshot_filename)

        # Save the screenshot
        cv2.imwrite(screenshot_full_path, frame)
        print(f"Screenshot saved: {screenshot_full_path}")

        # Increment the screenshot count
        screenshot_count += 1

    frames_processed += 1

    key = cv2.waitKey(1)
    if key == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

Screenshot saved: C:\CODING\19052013_MMTU_CD\ObjectDetection-YOLOv8-SSDMobileNet\ScreenShots-SSDMobileNet\screenshot_20240103145833_0.png
Screenshot saved: C:\CODING\19052013_MMTU_CD\ObjectDetection-YOLOv8-SSDMobileNet\ScreenShots-SSDMobileNet\screenshot_20240103145838_1.png
Screenshot saved: C:\CODING\19052013_MMTU_CD\ObjectDetection-YOLOv8-SSDMobileNet\ScreenShots-SSDMobileNet\screenshot_20240103145843_2.png
Screenshot saved: C:\CODING\19052013_MMTU_CD\ObjectDetection-YOLOv8-SSDMobileNet\ScreenShots-SSDMobileNet\screenshot_20240103145848_3.png
Screenshot saved: C:\CODING\19052013_MMTU_CD\ObjectDetection-YOLOv8-SSDMobileNet\ScreenShots-SSDMobileNet\screenshot_20240103145853_4.png
Screenshot saved: C:\CODING\19052013_MMTU_CD\ObjectDetection-YOLOv8-SSDMobileNet\ScreenShots-SSDMobileNet\screenshot_20240103145858_5.png
Screenshot saved: C:\CODING\19052013_MMTU_CD\ObjectDetection-YOLOv8-SSDMobileNet\ScreenShots-SSDMobileNet\screenshot_20240103145903_6.png
Screenshot saved: C:\CODING\190520

## UAV Surveillance Videosu

Bu kısımda video'nun path'i doğru verilmezse kod hata verebilir. Kod farklı bir bilgisayarda çalıştırıldığında path değişebilir! 

**Klavyede q tuşuna basılması dahilinde video kapanacaktır!**

In [5]:
video_path = r"C:\CODING\Python.PROJECTS\19052013_MMTU_CD\ObjectDetection-YOLOv8-SSDMobileNet\data\UAV-surveillance.mp4"

In [6]:
import cv2

def play_video(video_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    total_seconds = total_frames / fps

    cv2.namedWindow('Video Player', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Video Player', width, height)

    start_time = cv2.getTickCount()

    while True:
        ret, frame = cap.read()

        if not ret:
            print("End of video")
            break

        elapsed_time = (cv2.getTickCount() - start_time) / cv2.getTickFrequency()
        current_seconds = int(elapsed_time)
        remaining_seconds = int(total_seconds - current_seconds)
        display_text = f"{current_seconds:.1f}/{total_seconds:.1f} - FPS: {fps:.1f}"

        cv2.putText(frame, display_text, (width - 320, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('Video Player', frame)

        if cv2.waitKey(int(1000 / fps)) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Videoyu oynat
play_video(video_path)

### **YOLOv8** Object Detection Performansı - UAV Surveillance Video

In [15]:
from ultralytics import YOLO
import cv2
import math
import torch
import time
import os
from datetime import datetime



screen_shot_path = r'C:\CODING\Python.PROJECTS\19052013_MMTU_CD\ObjectDetection-YOLOv8-SSDMobileNet\ScreenShots\UAVSurveillance-YOLOv8'



cap = cv2.VideoCapture(video_path)
cap.set(3, 2000)
cap.set(4, 1000)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model
model = YOLO("yolo-Weights/yolov8n.pt")
model.to(device)

# object classes
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"]

# Initialize variables for FPS calculation and screenshot timing
start_time = time.time()
frames_processed = 0
screenshot_interval = 1  # İstenilen her kaç saniyede bir ekran görüntüsü alınacağını belirtir
screenshot_count = 0
screenshot_time = 0


while True:
    success, img = cap.read()

    if not success:
        print("Error reading video stream.")
        break

    # Perform YOLO object detection
    results = model(img, stream=True)

    # Calculate FPS
    frames_processed += 1
    elapsed_time = time.time() - start_time
    fps = frames_processed / elapsed_time

    # Display FPS on the frame
    cv2.putText(img, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Draw bounding boxes and class names
    for r in results:
        boxes = r.boxes

        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

            confidence = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])

            org = [x1, y1]
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = (255, 0, 0)
            thickness = 2

            cv2.putText(img, classNames[cls], org, font, fontScale, color, thickness)

    # Display the frame
    cv2.imshow("YOLO Object Detection", img)

    # Check if it's time to take a screenshot
    if time.time() - screenshot_time >= screenshot_interval:
        screenshot_time = time.time()

        # Create the screenshot path if it doesn't exist
        if not os.path.exists(screen_shot_path):
            os.makedirs(screen_shot_path)

        # Generate a filename for the screenshot using the current timestamp and count
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        screenshot_filename = f"screenshot_{timestamp}_{screenshot_count}.png"
        screenshot_full_path = os.path.join(screen_shot_path, screenshot_filename)

        # Save the screenshot
        cv2.imwrite(screenshot_full_path, img)
        print(f"Screenshot saved: {screenshot_full_path}")

        # Increment the screenshot count
        screenshot_count += 1

    if cv2.waitKey(1) == ord('q'):
        break


# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()



0: 352x640 19 cars, 3 buss, 2 trucks, 20.6ms
Speed: 3.2ms preprocess, 20.6ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)
Screenshot saved: C:\CODING\Python.PROJECTS\19052013_MMTU_CD\ObjectDetection-YOLOv8-SSDMobileNet\ScreenShots\UAVSurveillance-YOLOv8\screenshot_20240103231630_0.png

0: 352x640 18 cars, 3 buss, 2 trucks, 15.9ms
Speed: 2.1ms preprocess, 15.9ms inference, 5.1ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 19 cars, 4 buss, 16.5ms
Speed: 3.5ms preprocess, 16.5ms inference, 5.2ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 18 cars, 3 buss, 1 truck, 15.9ms
Speed: 4.9ms preprocess, 15.9ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 21 cars, 3 buss, 1 truck, 17.3ms
Speed: 3.8ms preprocess, 17.3ms inference, 3.4ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 18 cars, 3 buss, 15.4ms
Speed: 3.5ms preprocess, 15.4ms inference, 2.1ms postprocess per image at shape (1, 3, 352, 640)


### **SSD Mobile Net** Object Detection Performansı - UAV Surveillance Video


In [17]:
import cv2
import time
import numpy as np

cap = cv2.VideoCapture(video_path)
cap.set(3, 1280) # 3 represents width 
cap.set(4, 720) # 4 represents hight
cap.set(10, 70) # 10 represents brightness

threshold = 0.55
nms_threshold = 0.1

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
total_seconds = total_frames / fps

start_time = cv2.getTickCount()  

classNames = []
classFile = r'ssd-nesne-tanima-main\ssd-nesne-tanima-main\coco.names'
with open(classFile, 'rt') as f:
    classNames = f.read().rsplit('\n')

# print(classNames) # classNames can be observed
    

configPath = r'ssd-nesne-tanima-main\ssd-nesne-tanima-main\ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
weightsPath = r'ssd-nesne-tanima-main\ssd-nesne-tanima-main\frozen_inference_graph.pb'

SSDMobileNet = cv2.dnn_DetectionModel(weightsPath, configPath)
SSDMobileNet.setInputSize(320, 320)
SSDMobileNet.setInputScale(1.0/127.5)
SSDMobileNet.setInputMean((127.5, 127.5, 127.5))
SSDMobileNet.setInputSwapRB(True)


# Initialize variables for FPS calculation and screenshot timing
start_time = time.time()
frames_processed = 0
screenshot_interval = 1  # İstenilen her kaç saniyede bir ekran görüntüsü alınacağını belirtir
screenshot_count = 0
screenshot_time = 0

while True:
    success, frame = cap.read()

    if not success:
        print("Error reading video stream.")
        break

    current_time = time.time()
    elapsed_time = current_time - start_time
    current_seconds = int(elapsed_time)

    fps_and_videoLen_text = f"FPS: {frames_processed / elapsed_time:.2f} | Time: {current_seconds:.1f}/{total_seconds:.1f}"

    cv2.putText(frame, fps_and_videoLen_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    classIds, confs, bbox = SSDMobileNet.detect(frame, confThreshold=threshold)
    bbox = list(bbox)
    confs = list(np.array(confs).reshape(1, -1)[0])
    confs = list(map(float, confs))

    indices = cv2.dnn.NMSBoxes(bbox, confs, threshold, nms_threshold)

    for i in indices:
        box = bbox[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        cv2.rectangle(frame, (x, y), (x + w, h + y), color=(0, 255, 0), thickness=2)
        cv2.putText(frame, classNames[classIds[i] - 1].upper(), (box[0] + 10, box[1] + 30),
                    cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("SSD Mobile Net Object Detection", frame)

    # Check if it's time to take a screenshot
    if current_time - screenshot_time >= screenshot_interval:
        screenshot_time = current_time

        # Create the screenshot path if it doesn't exist
        screen_shot_path = r'C:\CODING\Python.PROJECTS\19052013_MMTU_CD\ObjectDetection-YOLOv8-SSDMobileNet\ScreenShots\UAVSurveillance-SSDMobileNet'
        if not os.path.exists(screen_shot_path):
            os.makedirs(screen_shot_path)

        # Generate a filename for the screenshot using the current timestamp and count
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        screenshot_filename = f"screenshot_{timestamp}_{screenshot_count}.png"
        screenshot_full_path = os.path.join(screen_shot_path, screenshot_filename)

        # Save the screenshot
        cv2.imwrite(screenshot_full_path, frame)
        print(f"Screenshot saved: {screenshot_full_path}")

        # Increment the screenshot count
        screenshot_count += 1

    frames_processed += 1

    key = cv2.waitKey(1)
    if key == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

Screenshot saved: C:\CODING\Python.PROJECTS\19052013_MMTU_CD\ObjectDetection-YOLOv8-SSDMobileNet\ScreenShots\UAVSurveillance-SSDMobileNet\screenshot_20240103231816_0.png
Screenshot saved: C:\CODING\Python.PROJECTS\19052013_MMTU_CD\ObjectDetection-YOLOv8-SSDMobileNet\ScreenShots\UAVSurveillance-SSDMobileNet\screenshot_20240103231817_1.png
Screenshot saved: C:\CODING\Python.PROJECTS\19052013_MMTU_CD\ObjectDetection-YOLOv8-SSDMobileNet\ScreenShots\UAVSurveillance-SSDMobileNet\screenshot_20240103231818_2.png
Screenshot saved: C:\CODING\Python.PROJECTS\19052013_MMTU_CD\ObjectDetection-YOLOv8-SSDMobileNet\ScreenShots\UAVSurveillance-SSDMobileNet\screenshot_20240103231819_3.png
Screenshot saved: C:\CODING\Python.PROJECTS\19052013_MMTU_CD\ObjectDetection-YOLOv8-SSDMobileNet\ScreenShots\UAVSurveillance-SSDMobileNet\screenshot_20240103231820_4.png
Screenshot saved: C:\CODING\Python.PROJECTS\19052013_MMTU_CD\ObjectDetection-YOLOv8-SSDMobileNet\ScreenShots\UAVSurveillance-SSDMobileNet\screenshot_2